# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Li  ->  J. Li  |  ['J. Li']
T. Henning  ->  T. Henning  |  ['T. Henning']
G. Perotti  ->  G. Perotti  |  ['G. Perotti']
A. Kamlah  ->  A. Kamlah  |  ['A. Kamlah']


C. Gieser  ->  C. Gieser  |  ['C. Gieser']
T. Henning  ->  T. Henning  |  ['T. Henning']
D. Semenov  ->  D. Semenov  |  ['D. Semenov']
T. Henning  ->  T. Henning  |  ['T. Henning']
Arxiv has 66 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2505.20393


extracting tarball to tmp_2505.20393...

 done.
Retrieving document from  https://arxiv.org/e-print/2505.20427


extracting tarball to tmp_2505.20427... done.


...

C. Gieser  ->  C. Gieser  |  ['C. Gieser']
T. Henning  ->  T. Henning  |  ['T. Henning']
D. Semenov  ->  D. Semenov  |  ['D. Semenov']


Found 92 bibliographic references in tmp_2505.21307/main_clean.bbl.
Retrieving document from  https://arxiv.org/e-print/2505.21470


extracting tarball to tmp_2505.21470... done.


T. Henning  ->  T. Henning  |  ['T. Henning']


Found 99 bibliographic references in tmp_2505.21470/Paper_2.bbl.
Error retrieving bib data for galliStructureKinematicsTaurus2019: 'author'
Error retrieving bib data for gudelXMMNewtonExtendedSurvey2007: 'author'


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.20427-b31b1b.svg)](https://arxiv.org/abs/2505.20427) | **Burned to ashes: How the thermal decomposition of refractory organics in the inner protoplanetary disc impacts the gas-phase C/O ratio**  |
|| A. Houge, et al. -- incl., <mark>T. Henning</mark>, <mark>G. Perotti</mark> |
|*Appeared on*| *2025-05-28*|
|*Comments*| *Accepted for publication in A&A. 18 pages, 14 figures, 1 table*|
|**Abstract**|            The largest reservoir of carbon in protoplanetary discs is stored in refractory organics, which thermally decompose into the gas-phase at the organics line, well interior to the water iceline. Because this region is so close to the host star, it is often assumed that the released gaseous material is rapidly accreted and plays little role in the evolution of the disc composition. However, laboratory experiments show that the thermal decomposition process is irreversible, breaking macromolecular refractory organics into simpler, volatile carbon-bearing compounds. As a result, unlike the iceline of other volatiles, which traps vapor inwards due to recondensation, the organics line remains permeable, allowing gaseous carbon to diffuse outward without returning to the solid phase. In this paper, we investigate how this process affects the disc composition, particularly the gas-phase C/H and C/O ratios, by incorporating it into a 1D evolution model for gas and solids, and assuming refractory organics dominantly decompose into C$_2$H$_2$. Our results show that this process allows this carbon-rich gas to survive well beyond the organics line (out to $7 \mathrm{~au}$ around a solar-mass star) and for much longer timescales, such that its abundance is increased by an order of magnitude. This has several implications in planet formation, notably by altering how the composition of solids and gas relate, and the fraction of heavy elements available to giant planets. In the framework of our model, refractory organics significantly influence the evolution of the gas-phase C/O ratio, which may help interpreting measurements made with Spitzer and JWST.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.21307-b31b1b.svg)](https://arxiv.org/abs/2505.21307) | **PRODIGE - envelope to disk with NOEMA: V. Low 12C/13C ratios for CH3OH and CH3CN in hot corinos**  |
|| L. A. Busch, et al. -- incl., <mark>C. Gieser</mark>, <mark>T. Henning</mark>, <mark>D. Semenov</mark> |
|*Appeared on*| *2025-05-28*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**|            The 12C/13C isotope ratio has been derived towards numerous cold clouds (20-50 K) and a couple protoplanetary disks and exoplanet atmospheres. However, direct measurements of this ratio in the warm gas (>100 K) around young low-mass protostars remain scarce, but are required to study its evolution during star and planet formation. We derived 12C/13C ratios from the isotopologues of the complex organic molecules (COMs) CH3OH and CH3CN in the warm gas towards seven Class 0/I protostellar systems to improve our understanding of the evolution of the 12C/13C ratios during star and planet formation. We used the data that were taken as part of the PRODIGE large program with the NOEMA at 1mm. The emission of CH3OH and CH3CN is spatially unresolved in the PRODIGE data (300au scale). Derived rotational temperatures exceed 100K, telling us that they trace the gas of the hot corino, where CH3CN probes hotter regions than CH3OH on average (290 K versus 180 K). The column density ratios between the 12C and 13C isotopologues, derived from LTE analysis, range from 4 to 30, thus, are significantly lower than the expected local ISM isotope ratio of about 68. Assuming that CH3CN and CH3OH may inherit the 12C/13C ratio from their precursor species, astrochemical models were conducted for the latter and compared with our observational results. We conclude that an enrichment in 13C in COMs at the earliest protostellar stages is likely inherited from the COMs' precursor species, whose 12C/13C ratios are set during the prestellar stage via isotopic exchange reactions. This also implies that low 12C/13C ratios observed at later evolutionary stages could at least partially be inherited. A final conclusion on 12C/13C ratios in protostellar environments requires improved observations to tackle current observational limitations and additional modelling efforts.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.21470-b31b1b.svg)](https://arxiv.org/abs/2505.21470) | **Changing disc compositions via internal photoevaporation II: M dwarf systems**  |
|| J. L. Lienert, B. Bitsch, <mark>T. Henning</mark> |
|*Appeared on*| *2025-05-28*|
|*Comments*| **|
|**Abstract**|            The chemical evolution of the inner regions of protoplanetary discs is a complex process. Several factors influence it, one being the inward drift and evaporation of volatile-rich pebbles. During the disc's evolution, its inner part is first enriched with evaporating water-ice, resulting in a low C/O ratio. Afterwards, C-rich gas from the outer disc is transported inwards. Consequently, the C/O ratio of the inner disc increases again after 2 Myr. Previously, we studied how internal photoevaporation influences these processes in discs around Sun-like stars. We now extend our study to lower-mass stars, where the time evolution of the disc's C/O ratio is different due to the closer-in position of the evaporation fronts and differences in disc mass, size and structure. Our simulations are carried out with the semi-analytical 1D disc model chemcomp, which includes viscous evolution and heating, pebble growth and drift, pebble evaporation and condensation, as well as a simple chemical partitioning model. We show that internal photoevaporation plays a major role in the evolution of protoplanetary discs: As for Sun-like stars, photoevaporation opens a gap, which stops inward drifting pebbles. In addition, volatile-rich gas from the outer disc is carried away by the photoevaporative winds. Consequently, the C/O ratio in the inner disc remains low, contradicting observations of discs around low-mass stars. Our model implies that young inner discs (< 2 Myr) should be O-rich and C-poor, while older discs (> 2 Myr) should be C-rich. The survival of discs to this age can be attributed to lower photoevaporation rates, which either originate from a large spread of observed X-ray luminosities or from the photoevaporation model used here, which likely overestimates the photoevaporation efficiency. A reduction of the latter brings the calculated elemental abundances into better agreement with observations.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.20432-b31b1b.svg)](https://arxiv.org/abs/2505.20432) | **Evolution of star clusters with initial bulk rotation via N-body simulations**  |
|| A. Bissekenov, et al. -- incl., <mark>A. Kamlah</mark> |
|*Appeared on*| *2025-05-28*|
|*Comments*| **|
|**Abstract**|            Young star clusters can inherit bulk rotation from the molecular clouds from which they have formed. This rotation can affect the long-term evolution of a star cluster and its constituent stellar populations. In this study, we aim to characterize the effects of different degrees of initial rotation on star clusters with primordial binaries. The simulations are performed using NBODY6++GPU. We find that initial rotation strongly affects the early evolution of star clusters. Rapidly rotating clusters show angular momentum transport from the inner parts to the outskirts, resulting in a core collapse. Angular momentum transport is accompanied by a highly elongated bar-like structure morphology. The effects of bulk rotation are reduced on the timescale of two-body relaxation. Rotating and non-rotating clusters experience changes in the direction of angular momentum near the dissolution and early evolution due to the tidal field, respectively. We present synthetic observations of simulated clusters for comparison with future observations in filters of Gaia, CSST, and HST. This work shows the effects of bulk rotation on systems with primordial binaries and could be used for the identification of rotation signatures in observed open clusters.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.20393-b31b1b.svg)](https://arxiv.org/abs/2505.20393) | **NEXUS: A Spectroscopic Census of Broad-line AGNs and Little Red Dots at $3\lesssim z\lesssim 6$**  |
|| M.-Y. Zhuang, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-05-28*|
|*Comments*| *26 pages, 17 figures, 4 tables. Submitted to ApJ*|
|**Abstract**|            We present a spectroscopic sample of 23 broad-line AGNs (BLAGNs) at $3\lesssim z\lesssim 6$ selected using F322W2+F444W NIRCam/WFSS grism spectroscopy of the central 100 ${\rm arcmin^2}$ area of the NEXUS survey. Among these BLAGNs, 15 are classified as Little Red Dots (LRDs) based on their rest-frame UV-optical spectral slopes and compact morphology. The number density of LRDs is $\sim 10^{-5}\,{\rm cMpc^{-3}}$, with a hint of declining towards the lower end of the probed redshift range. These BLAGNs and LRDs span broad H$\alpha$ luminosities of $\sim 10^{42.2}-10^{43.7}\,{\rm erg\,s^{-1}}$, black hole masses of $\sim 10^{6.3}-10^{8.4}\,M_\odot$, and Eddington ratios of $\sim 0.1-1$ (median value 0.4), though the black hole mass and Eddington ratio estimates carry large systematic uncertainties. Half of the LRDs show strong Balmer absorption, suggesting high-density gas surrounding the line-emitting region. We detect extended (hundreds of parsec) rest-frame UV-optical emission from the host galaxy in the majority of these LRDs, which contributes significantly or even dominantly to their total UV emission. This host emission largely accounts for the peculiar UV upturn of the LRD spectral energy distribution. We also measure the small-scale ($\lesssim 1\,{\rm cMpc}$) clustering of these BLAGNs and LRDs by cross-correlating with a photometric galaxy sample. Extrapolating the power-law two-point correlation function model to large linear scales, we infer a linear bias of $3.30_{-2.04}^{+2.88}$ and typical halo masses of a few $\times 10^{11}\,h^{-1}M_\odot$ for BLAGNs at the sample median redshift of $z\sim 4.5$. However, the inferred linear bias and halo masses of LRDs, while formally consistent with those for BLAGNs at $\sim 1.5\sigma$, appear too large to be compatible with their space density, suggesting LRDs may have strong excess clustering on small scales.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2505.20427/./CO_radius_c2h2_1ms_alpha1e3_tr_vs_notr.png', 'tmp_2505.20427/./CH_radius_c2h2_1ms_alpha1e3_with_or_no_trsfr.png', 'tmp_2505.20427/./SigmaD_SigmaG_organics_c2h2_1Msun_alpha1e3.png']
copying  tmp_2505.20427/./CO_radius_c2h2_1ms_alpha1e3_tr_vs_notr.png to _build/html/
copying  tmp_2505.20427/./CH_radius_c2h2_1ms_alpha1e3_with_or_no_trsfr.png to _build/html/
copying  tmp_2505.20427/./SigmaD_SigmaG_organics_c2h2_1Msun_alpha1e3.png to _build/html/
exported in  _build/html/2505.20427.md
    + _build/html/tmp_2505.20427/./CO_radius_c2h2_1ms_alpha1e3_tr_vs_notr.png
    + _build/html/tmp_2505.20427/./CH_radius_c2h2_1ms_alpha1e3_with_or_no_trsfr.png
    + _build/html/tmp_2505.20427/./SigmaD_SigmaG_organics_c2h2_1Msun_alpha1e3.png
found figures ['tmp_2505.21307/./figs/PD/ch3oh_IRAS4B_on.png', 'tmp_2505.21307/./figs/PD/ch3oh.13c_IRAS4B_on.png', 'tmp_2505.21307/./figs/PD/ch3cn_IRAS4B_on.png', 'tmp_2505.21307/./figs/PD/ch3cn.13c1_IRAS4B_on.png', 'tmp_2505.21307/./figs

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Adrien}[1]{\textcolor{red}{#1}}$
$\newcommand{\edited}[1]{{#1}}$</div>



<div id="title">

# Burned to ashes:\ How the thermal decomposition of refractory organics in the inner protoplanetary disc impacts the gas-phase C/O ratio

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2505.20427-b31b1b.svg)](https://arxiv.org/abs/2505.20427)<mark>Appeared on: 2025-05-28</mark> -  _Accepted for publication in A&A. 18 pages, 14 figures, 1 table_

</div>
<div id="authors">

A. Houge, et al. -- incl., <mark>T. Henning</mark>, <mark>G. Perotti</mark>

</div>
<div id="abstract">

**Abstract:** The largest reservoir of carbon in protoplanetary discs is stored in refractory organics, which thermally decompose into the gas-phase at the organics line, well interior to the water iceline. Because this region is so close to the host star, it is often assumed that the released gaseous material is rapidly accreted and plays little role in the evolution of the disc composition. However, laboratory experiments show that the thermal decomposition process is irreversible, breaking macromolecular refractory organics into simpler, volatile carbon-bearing compounds. As a result, unlike the iceline of other volatiles, which traps vapor inwards due to recondensation, the organics line remains permeable, allowing gaseous carbon to diffuse outward without returning to the solid phase. In this paper, we investigate how this process affects the disc composition, particularly the gas-phase C/H and C/O ratios, by incorporating it into a 1D evolution model for gas and solids, and assuming refractory organics dominantly decompose into $C_2$ $H_2$ . Our results show that this process allows this carbon-rich gas to survive well beyond the organics line (out to $7 \mathrm{ au}$ around a solar-mass star) and for much longer timescales, such that its abundance is increased by an order of magnitude. This has several implications in planet formation, notably by altering how the composition of solids and gas relate, and the fraction of heavy elements available to giant planets. In the framework of our model, refractory organics significantly influence the evolution of the gas-phase C/O ratio, which may help $\edited{interpreting}$ measurements made $\edited{with}$ Spitzer and JWST.

</div>

<div id="div_fig1">

<img src="tmp_2505.20427/./CO_radius_c2h2_1ms_alpha1e3_tr_vs_notr.png" alt="Fig10" width="100%"/>

**Figure 10. -** Gas-phase (left panel) and solid-phase (right panel) C/O ratios as functions of distance from a solar-mass star for a disc characterised by $\alpha = 10^{-3}$. Blue lines represent a standard model that excludes the thermal decomposition of refractory organics \citep[e.g.,][]{mah2023close}, while red lines show the results from our model, including that effect. The horizontal dashed grey line represents the initial C/O ratio. The sublimation lines of the main species contributing to the C/O ratio are indicated with vertical lines. From this plot, we see that the thermal decomposition of refractory organics significantly influences the gas-phase and solid-phase C/O ratio in the disc. The gas-phase C/O ratio is generally much higher when the thermal decomposition of refractory organics is included. (*fig:CO_vs_radius_tr_vs_notr*)

</div>
<div id="div_fig2">

<img src="tmp_2505.20427/./CH_radius_c2h2_1ms_alpha1e3_with_or_no_trsfr.png" alt="Fig2" width="100%"/>

**Figure 2. -** Gas-phase C/H ratio (normalised by the ratio at $t_0$) as a function of distance from a solar-mass star for a disc characterised by $\alpha = 10^{-3}$. Blue lines represent a standard model that excludes the thermal decomposition of refractory organics \citep[e.g.,][]{mah2023close}, while red lines show the results from our model, including that effect. The sublimation lines of the main species contributing to the C/O ratio are indicated with vertical lines. From this plot, we see that the thermal decomposition of refractory organics re-distributes gaseous carbon-bearing species through the disc, hence significantly influences the gas-phase C/H ratio. (*fig:CH_tr_vs_notr*)

</div>
<div id="div_fig3">

<img src="tmp_2505.20427/./SigmaD_SigmaG_organics_c2h2_1Msun_alpha1e3.png" alt="Fig9" width="100%"/>

**Figure 9. -** Gas surface density (top) and solid surface density (bottom) of the main chemical species that contribute to the C/O ratio at $0.1$, $1$, and $5 \mathrm{ Myr}$. We ran this simulation with $M_{*} = 1 \mathrm{ M_\odot}$ and $\alpha = 10^{-3}$. The total (dust or gas) surface density is represented as a dark solid line in each row. Vertical dashed lines indicate the sublimation lines of the displayed species in their respective color, using $T_\mathrm{sub, CO} = 20 \mathrm{ K}$, $T_\mathrm{sub, CH_4} = 30 \mathrm{ K}$, $T_\mathrm{sub, CO_2} = 70 \mathrm{ K}$, $T_\mathrm{sub, C_2H_2} = 70 \mathrm{ K}$, $T_\mathrm{sub, H_2O} = 150 \mathrm{ K}$, and $T_\mathrm{sub, refractory} = 350 \mathrm{ K}$(see Table \ref{tab:partition_abundances}). (*fig:SigmaD_SigmaG*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2505.20427"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\mmc}{CH_3SH\xspace}$
$\newcommand{\met}{CH_3OH\xspace}$
$\newcommand{\metc}{CH_3CN\xspace}$
$\newcommand{\metctr}{CH_3^{13}CN\xspace}$
$\newcommand{\et}{C_2H_5OH\xspace}$
$\newcommand{\etc}{C_2H_5CN\xspace}$
$\newcommand{\vc}{C_2H_3CN\xspace}$
$\newcommand{\dme}{CH_3OCH_3\xspace}$
$\newcommand{\mf}{CH_3OCHO\xspace}$
$\newcommand{\mic}{CH_3NCO\xspace}$
$\newcommand{\ad}{CH_3CHO\xspace}$
$\newcommand{\fmm}{NH_2CHO\xspace}$
$\newcommand{\2}{_2}$
$\newcommand{\3}{_3}$
$\newcommand{\5}{_5}$
$\newcommand{\7}{_7}$
$\newcommand{\ct}{^{13}}$
$\newcommand{\cd}{^{12}}$
$\newcommand{\ratio}{^{12}C / ^{13}C\xspace}$
$\newcommand{\oratio}{^{16}O / ^{18}O\xspace}$
$\newcommand{\odh}{^{18}}$
$\newcommand{\scm}{cm^{-2}\xspace}$
$\newcommand{\kms}{ km s^{-1}\xspace}$</div>



<div id="title">

# PRODIGE -- envelope to disk with NOEMA: V. Low $\ratio$ ratios for $\met$ and $\metc$ in hot corinos

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2505.21307-b31b1b.svg)](https://arxiv.org/abs/2505.21307)<mark>Appeared on: 2025-05-28</mark> -  _Accepted for publication in A&A_

</div>
<div id="authors">

L. A. Busch, et al. -- incl., <mark>C. Gieser</mark>, <mark>T. Henning</mark>, <mark>D. Semenov</mark>

</div>
<div id="abstract">

**Abstract:** The $\ratio$ isotope ratio has been derived towards numerous cold clouds ( $\sim$ 20--50 K) and a couple protoplanetary disks and exoplanet atmospheres.  However, direct measurements of this ratio in the warm gas ( $>$ 100 K) around young low-mass protostars remain scarce, but are required to study its evolution during star and planet formation. We aim to derive $\ratio$ ratios from the isotopologues of the complex organic molecules (COMs) $\met$ and $\metc$ in the warm gas towards seven Class 0/I protostellar systems to improve our understanding of the evolution of the $\ratio$ ratios during star and planet formation. We used the data that were taken as part of the PRODIGE (PROtostars \& DIsks: Global Evolution) large program with the Northern Extended Millimeter Array (NOEMA) at 1 mm. The $\ct$ C isotopologue of $\met$ was detected towards seven sources of the sample, the ones of $CH_3$ CN towards six. The emission spectra were analysed by deriving synthetic spectra and population diagrams assuming conditions of local thermodynamic equilibrium (LTE). The emission of $\met$ and $\metc$ is spatially unresolved in the PRODIGE data with a resolution of $\sim$ 1 $\arcsec$ ( $\sim$ 300 au) for the seven targeted systems. Rotational temperatures derived from both COMs exceed 100 K, telling us that they trace the gas of the hot corino, where $\metc$ probes hotter regions than $\met$ on average (290 K versus 180 K). The column density ratios between the $\cd$ C and $\ct$ C isotopologues range from 4 to 30, thus, are lower by factors of a few up to an order of magnitude than the expected local ISM isotope ratio of $\sim$ 68. We conducted astrochemical models to understand the origins of the observed low ratios. We studied potential precursor molecules of $\met$ and $\metc$ , since the model does not include COMs, assuming that the ratio is transferred in reactions from the precursors to the COMs. The model predicts $\ratio$ ratios close to the ISM value for CO and $H_2$ CO, precursors of $CH_3$ OH, in contrast to our observational results. For the potential precursors of $CH_3$ CN (CN, HCN, and HNC), the model predicts low $\ratio$ ratios close to the protostar ( $<300$ au), hence, they may also be expected for $\metc$ . Our results show that an enrichment in $\ct$ C in COMs at the earliest protostellar stages is likely inherited from the COMs' precursor species, whose $\ratio$ ratios are set during the prestellar stage via isotopic exchange reactions. This also implies that low $\ratio$ ratios observed at later evolutionary stages such as protoplanetary disks and exoplanetary atmospheres could at least partially be inherited. A final conclusion on $\ratio$ ratios in protostellar environments requires observations at higher angular and spectral resolution that simultaneously cover a broad bandwidth, to tackle current observational limitations, and additional modelling efforts.

</div>

<div id="div_fig1">

<img src="tmp_2505.21307/./figs/PD/ch3oh_IRAS4B_on.png" alt="Fig12.1" width="20%"/><img src="tmp_2505.21307/./figs/PD/ch3oh.13c_IRAS4B_on.png" alt="Fig12.2" width="20%"/><img src="tmp_2505.21307/./figs/PD/ch3cn_IRAS4B_on.png" alt="Fig12.3" width="20%"/><img src="tmp_2505.21307/./figs/PD/ch3cn.13c1_IRAS4B_on.png" alt="Fig12.4" width="20%"/><img src="tmp_2505.21307/./figs/PD/ch3cn.13c2_IRAS4B_on.png" alt="Fig12.5" width="20%"/>

**Figure 12. -** Population diagrams for $\met$, $\ct$$\met$, $\met$c, $\ct$$\met$c, and $\met$ctr towards IRAS 4B. Observed data points are shown with teal circles as indicated in the top right corner of the left panel while the modelled data points from Weeds are shown orange squares. No corrections are applied in the left panel while in the right panel corrections for opacity and contamination by other molecules have been considered for the observed and modelled populations.
    The results of the linear fit to the observed data points are shown in the right panels.  (*pd:iras4b*)

</div>
<div id="div_fig2">

<img src="tmp_2505.21307/./figs/Maps/IRAS2A.png" alt="Fig5" width="100%"/>

**Figure 5. -** Integrated intensity maps (in K $\kms$) using two transitions for both isotopologues of $\met$, the main $\met$c isotopologue, and one transition for each $\ct$ C isotopologue towards IRAS 2A. Respective rest frequencies in GHz are shown in the bottom right corner, the beam (half power beam width, HPBW) on the bottom left. Multiplication factors (e.g. $\times$3, next to the species name) were applied to some species and transitions to be able to use the same intensity scale for all panels. The yellow dashed contour is at 5$\sigma$, where $\sigma$ is the rms noise level (in K $\kms$) and is written in the top right corner. (*maps:iras2a*)

</div>
<div id="div_fig3">

<img src="tmp_2505.21307/./figs/data_vis.png" alt="Fig22" width="100%"/>

**Figure 22. -** Overview of the PRODIGE data towards IRAS 4B and presentation of the spectral-line analysis . _Panel a:_ Integrated intensity map (in K $\kms$) for the $\met$c$12_3-11_3$ transition at 220.708 GHz. The beam (half power beam width, HPBW) is shown in the bottom left. The dashed yellow contour is at 5$\sigma$, where $\sigma$ is the rms noise level (in K $\kms$) and is written in the top right corner. _Panel b:_ Observed spectra towards the peak position, which is at $(\Delta\alpha,\Delta\beta)=(0$\arcsec$,0$\arcsec$)$(black cross in panel a), of $\met$c at low ($\sim$2 MHz, black) and high ($\sim$62.5 kHz, teal) spectral resolution, where the latter was shifted by 8 K for visualisation. The low spectral resolution data were analysed in this work (see Sect. \ref{s:method}). The modelled $\met$c spectra from Weeds are shown in red (low resolution) and orange (high resolution), where the same input parameters were used for both. _Panel c:_ Population diagram for $\met$c. Teal circles show the observed data points while the modelled data points from Weeds are shown with orange squares (see Sect. \ref{ss:caveats} for possible origins of differences between the two). No corrections are applied in the left panel while in the right panel corrections for opacity and contamination by other molecules have been considered (negligible effect in this case) and only transitions with optical depths $<$ 0.5 (i.e. all transitions in this case) are shown.
    The results of the linear fit to the observed data points are shown in the right panels. (*fig:overview*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2505.21307"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Changing disc compositions via internal photoevaporation II: \ M dwarf systems

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2505.21470-b31b1b.svg)](https://arxiv.org/abs/2505.21470)<mark>Appeared on: 2025-05-28</mark> - 

</div>
<div id="authors">

J. L. Lienert, B. Bitsch, <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** The chemical evolution of the inner regions of protoplanetary discs is a complex process. It is influenced by several factors, one being the inward drift and evaporation of volatile-rich pebbles that can enrich the inner disc with vapour. During the evolution of the disc, its inner part is first enriched with evaporating water-ice, resulting in a low C/O ratio. Afterwards, carbon-rich gas from the outer disc, originating from the evaporation of CO, $CO_2$ and $CH_4$ ice, is viscously transported inwards, while at the same time, the supply of water-rich pebbles ceases and the water vapour in the inner disc is accreted onto the star. Consequently, the C/O ratio of the inner disc increases again after $2   \text{Myr}$ . Previously, we studied how internal photoevaporation influences the chemical composition and evolution of discs around Sun-like stars by carrying away gas and opening gaps that block inward drifting pebbles. We now extend our study to lower-mass stars ( $M_{\star} = 0.1 - 0.5   \text{M}_{\odot}$ ), where the time evolution of the disc's C/O ratio is different due to the closer-in position of the evaporation fronts and differences in disc mass, size and structure. Our simulations are carried out with a semi-analytical 1D disc model. The code \texttt{chemcomp} includes viscous evolution and heating, pebble growth and drift, pebble evaporation and condensation, as well as a simple chemical partitioning model for the disc. We show that internal photoevaporation plays a major role in the evolution of protoplanetary discs and their chemical composition: As for solar-mass stars, photoevaporation opens a gap, which stops inward drifting pebbles. They then cannot contribute to the volatile content of the gas in the inner disc any more. In addition, volatile-rich gas from the outer disc, originating from evaporated CO, $CO_2$ or $CH_4$ ice, is carried away by the photoevaporative winds. Consequently, the C/O ratio in the inner disc remains low, contradicting observations of the composition of discs around low-mass stars. Our model implies that inner discs at young ages ( $< 2   \text{Myr}$ ) should be oxygen-rich and carbon-poor, while older discs ( $> 2   \text{Myr}$ ) should be carbon-rich. The survival of discs to this age can be attributed to lower photoevaporation rates. These lower rates could either originate from the large spread of observed X-ray luminosities or from the photoevaporation model used in this study \citep[based on][]{picognaDispersalProtoplanetaryDiscs2021b} , which likely overestimates the photoevaporation efficiency at a given X-ray luminosity, leading to discrepancies with the observed C/O ratios in discs around low-mass stars. A reduction of the photoevaporation rate brings the calculated elemental abundances into better agreement with observations.

</div>

<div id="div_fig1">

<img src="tmp_2505.21470/./Plots/gas_surface_density_and_C_to_O_ratio_with_photoevap_alpha_1e-4.png" alt="Fig4" width="100%"/>

**Figure 4. -** Disc evolution for a viscous disc with internal photoevaporation due to X-rays, using the nominal photoevaporative mass loss rates from table \ref{table:fit_parameters}. The host star masses vary from $0.5   \text{M}_{\odot}$(on the left) to $0.1   \text{M}_{\odot}$(on the right). **Top:** Gas surface density as a function of disc radius, time evolution is shown in colour - from black, which corresponds to $0   \text{Myr}$, to dark red, which corresponds to $10   \text{Myr}$. **Bottom:** Gaseous C/O ratio as a function of disc radius and time (colour-coded). The evaporation lines for different molecules are given as grey dashed lines. Note that the C/O ratio is calculated from number densities and that we, by definition, have no specified C/O ratio in the gas phase beyond the CO evaporation front. We use our standard parameters for this simulation, as given in table \ref{table:simulation_parameters}. (*fig:overview_disc_with_PE*)

</div>
<div id="div_fig2">

<img src="tmp_2505.21470/./Plots/gas_surface_density_and_C_to_O_ratio_no_photoevap_alpha_1e-4.png" alt="Fig8" width="100%"/>

**Figure 8. -** Disc evolution for a viscous disc without internal photoevaporation. The host star masses vary from $0.5   \text{M}_{\odot}$(on the left) to $0.1   \text{M}_{\odot}$(on the right). **Top:** Gas surface density as a function of disc radius and time. **Bottom:** Gaseous C/O ratio as a function of disc radius and time. Colour coding, plotting and simulation parameters as in figure \ref{fig:overview_disc_with_PE}. (*fig:overview_viscous_disc*)

</div>
<div id="div_fig3">

<img src="tmp_2505.21470/./Plots/X_ray_lum_vs_mass.png" alt="Fig2" width="100%"/>

**Figure 2. -** X-ray luminosity, derived from the measured X-ray flux, as a function of stellar mass for all detected \ac{XEST} sources, re-plotted from gudelXMMNewtonExtendedSurvey2007 using their original data. The distance to Taurus is estimated with $140   \text{pc}$, although we note that Taurus consists of subgroups with varying distances  (galliStructureKinematicsTaurus2019) . The flux of stars with more than one measurement has been averaged to retrieve only one data point for the plot. The circle, star and diamond symbols define the stellar class, as defined for \ac{XEST} sources, and the straight line gives a linear regression for the logarithmic values, being described by the following equation, $\log(L_{\text{X}}) = 1.54 \log(M_{\star}) + 30.31$. Synthetic values from our simulations from sections \ref{ssec:Reduced_photoevaporation_rate} and \ref{ssec:Disc_lifetimes} are added as purple coloured squares, with the colour indicating the gap opening time of the disc, $t_{\text{gap}}$. (*fig:guedel_fig_12*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2505.21470"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

504  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
